In [ ]:
from model import *
my_model = mymodel().cuda()
my_model.eval()
my_model.load_state_dict(torch.load('./gen_model/base/2/model'))
print('ok') 

In [4]:
from torch.utils.data import Dataset, DataLoader
class e2eDataset(Dataset):
    def __init__(self, csv_file, tokenizer):
        """
        Args:
            csv_file (string): csv 파일의 경로
        """
        self.dataset = pd.read_csv(csv_file)
        self.columns = self.dataset.columns
        self.conditions = self.dataset[self.columns[0]]
        self.sentences = self.dataset[self.columns[1]]
        self.tokenizer = tokenizer
        
        self.typ_list = {}
        for k in range(len(self.conditions)):
            cond_set = self.conditions[k].split(',')
            for m in range(len(cond_set)):
                cond_set[m] = cond_set[m].strip()
                pos = cond_set[m].index('[')
                if cond_set[m][:pos] in self.typ_list.keys():
                    self.typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
                else:            
                    self.typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}        

    def __len__(self):
        return len(self.conditions)

    def __getitem__(self, idx):
        cond = self.conditions[idx]
        cond_set = cond.split(',')
        condition_string = ''
        for m in range(len(cond_set)):
            cond_set[m] = cond_set[m].strip()
            pos = cond_set[m].index('[')
            
            condition_string += '<' + cond_set[m][:pos] + '>' + cond_set[m][pos+1:-1] + ' '
        
        sen = self.sentences[idx]
        input_string = condition_string + '<START>'
        input_ids = torch.tensor(self.tokenizer.encode(input_string, add_special_tokens=True))
        
        input_len = len(input_ids)

        return input_ids, sen, condition_string


In [ ]:
# e2e_dataset_test = e2eDataset(csv_file='dataset/testset_w_refs.csv', tokenizer=my_model.tokenizer)
# e2e_dataset_test2 = e2eDataset(csv_file='dataset/testset.csv', tokenizer=my_model.tokenizer)
# e2e_dataset_train = e2eDataset(csv_file='dataset/trainset.csv', tokenizer=my_model.tokenizer)
e2e_dataset = e2eDataset(csv_file='dataset/devset.csv', tokenizer=my_model.tokenizer)
dataloader = DataLoader(e2e_dataset, batch_size=1, shuffle=False, num_workers=4)   

In [ ]:
len(e2e_dataset_train), len(e2e_dataset), len(e2e_dataset_test)

In [ ]:
for k, v in e2e_dataset_train.typ_list.items():
    print(k, len(v))
e2e_dataset_train.typ_list['area']

In [ ]:
# x = my_model.tokenizer.additional_special_tokens_ids
x = [50258, 50259, 50263, 50260, 50261, 50264, 50265, 50262]
print(x)
print(my_model.tokenizer.all_special_tokens)
for i in range(len(x)):
    print(my_model.tokenizer.decode([x[i]]))

In [ ]:
from tqdm import tqdm
same_condition = []
ref_sentences = []
input_ids_list = []
pre_condition_string = ''
start = 0
for i_batch, sample_batched in tqdm(enumerate(dataloader)):
    sen = sample_batched[1][0]
#     print(i_batch, sen)
    condition_string = sample_batched[2]  
    input_ids = sample_batched[0].squeeze(0).cuda()
        
    if start == 0 or condition_string == pre_condition_string:      
        if start == 0:
            input_ids_list.append(input_ids)
        same_condition.append(sen)        
        pre_condition_string = condition_string
        start += 1
    else:   
        input_ids_list.append(input_ids)
        ref_sentences.append(same_condition)
        pre_condition_string = condition_string
        same_condition = [sen]
        start += 1

#     if i_batch == 30:
#         break            
ref_sentences.append(same_condition)

In [ ]:
len(ref_sentences), len(input_ids_list)
# input_ids_list[0]

In [ ]:
import time
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
max_len = 50
start_time = time.time()
bleu_score = 0
bleu_1 = 0

f_dev = open('./predictions/base/f_dev_2.txt', 'w')
f_pred = open('./predictions/base/f_pred_2.txt', 'w')

for k in range(len(ref_sentences)):
    input_ids = input_ids_list[k]
    input_len = len(input_ids)

    ori_tokens = []
    for m in range(len(ref_sentences[k])):
        f_dev.write(ref_sentences[k][m]+'\n')
        ori_tokens.append(word_tokenize(ref_sentences[k][m]))
    if k < len(ref_sentences)-1:
        f_dev.write('\n')
    
    for _ in range(max_len):
        model_out = my_model.model_feeding(input_ids) # (batch, seq_len, emb_dim)
        pred_idx = model_out.argmax(1)[-1]        
        if pred_idx == my_model.tokenizer.eos_token_id:
            break            
        input_ids = torch.cat((input_ids, pred_idx.unsqueeze(0)), 0)        
    
    out_sen = my_model.tokenizer.decode(input_ids[input_len:])
    f_pred.write(out_sen+'\n')
    
#     print(ref_sentences[k])
#     print(out_sen)    
    
    out_tokens = word_tokenize(out_sen)
    
    bleu_1_score = sentence_bleu(ori_tokens, out_tokens, weights=(1, 0, 0, 0))
    bleu_2_score = sentence_bleu(ori_tokens, out_tokens, weights=(0.5, 0.5, 0, 0))
    bleu_3_score = sentence_bleu(ori_tokens, out_tokens, weights=(0.33, 0.33, 0.33, 0))
    bleu_4_score = sentence_bleu(ori_tokens, out_tokens, weights=(0.25, 0.25, 0.25, 0.25))
    
    bleu_1 += bleu_1_score

    bleu_score += min(1, len(out_tokens)/len(ori_tokens))*((bleu_1_score*bleu_2_score*bleu_3_score*bleu_4_score)**(0.25))    
    
#     print("time: {}".format(time.time()-start_time))
#     print('')
f_dev.close()
f_pred.close()
    
print("BLEU score: {}".format(bleu_score/len(ref_sentences)*100))
print("BLEU1 score: {}".format(bleu_1/len(ref_sentences)*100))

In [ ]:
BLEU score: 71.18825887456514
BLEU1 score: 88.76109911863252

./measure_scores.py ../predictions/base/f_dev_6.txt ../predictions/base/f_pred_6.txt
./measure_scores.py ../predictions/base2/f_dev_2.txt ../predictions/base2/f_pred_2.txt
./measure_scores.py ../predictions/large/f_dev_5.txt ../predictions/large/f_pred_5.txt
./measure_scores.py ../predictions/f_dev_2.txt ../predictions/large2/f_pred_2.txt

### Large
#### my_output_1
BLEU: 0.6826
NIST: 8.4243
METEOR: 0.4557
ROUGE_L: 0.7032
CIDEr: 2.1234

#### my_output_2
BLEU: 0.7228
NIST: 8.5241
METEOR: 0.4851
ROUGE_L: 0.7461
CIDEr: 2.4645

#### my_output_3
BLEU: 0.7035
NIST: 8.5937
METEOR: 0.4700
ROUGE_L: 0.7252
CIDEr: 2.3310

#### my_output_4
BLEU: 0.6738
NIST: 8.4018
METEOR: 0.4576
ROUGE_L: 0.7075
CIDEr: 2.2172

#### my_output_5
BLEU: 0.6927
NIST: 8.4429
METEOR: 0.4662
ROUGE_L: 0.7180
CIDEr: 2.2729

### base
#### my_output_2
BLEU: 0.6812
NIST: 8.5491
METEOR: 0.4442
ROUGE_L: 0.7036
CIDEr: 2.1261

#### my_output_6
BLEU: 0.6655
NIST: 8.4830
METEOR: 0.4475
ROUGE_L: 0.6992
CIDEr: 2.1077
    
#### my_output_final
BLEU: 0.6529
NIST: 8.3116
METEOR: 0.4430
ROUGE_L: 0.6842
CIDEr: 1.9766

### Pragmatically Informative Text Generation
BLEU 68.60
NIST 8.73
METEOR 45.25
R-L 70.82
CIDEr 2.37

## Total model inference

In [5]:
# from model_large import *
from model import *
from tqdm import tqdm
import time
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
max_len = 70

my_model = mymodel().cuda()
my_model.eval()

for i in range(1, 9):
    model_name = './gen_model/base2_sample_30/'+str(i)+'/model'
    my_model.load_state_dict(torch.load(model_name))
    print('ok') 
    if i == 1:
#         e2e_dataset = e2eDataset(csv_file='dataset/devset.csv', tokenizer=my_model.tokenizer)
        e2e_dataset = e2eDataset(csv_file='dataset/testset_w_refs.csv', tokenizer=my_model.tokenizer)
        dataloader = DataLoader(e2e_dataset, batch_size=1, shuffle=False, num_workers=4)
        same_condition = []
        ref_sentences = []
        input_ids_list = []
        pre_condition_string = ''
        start = 0
        for i_batch, sample_batched in tqdm(enumerate(dataloader)):
            sen = sample_batched[1][0]
            condition_string = sample_batched[2]  
            input_ids = sample_batched[0].squeeze(0).cuda()

            if start == 0 or condition_string == pre_condition_string:      
                if start == 0:
                    input_ids_list.append(input_ids)
                same_condition.append(sen)        
                pre_condition_string = condition_string
                start += 1
            else:   
                input_ids_list.append(input_ids)
                ref_sentences.append(same_condition)
                pre_condition_string = condition_string
                same_condition = [sen]
                start += 1
        ref_sentences.append(same_condition)    

    bleu_score = 0
    bleu_1 = 0

#     f_dev = open('./predictions/testset/large2/f_dev_'+str(i)+'.txt', 'w')
#     f_pred = open('./predictions/devset/base4/f_pred_'+str(i)+'.txt', 'w')
    f_pred = open('./predictions/joosung2/testset/base2_sample30/f_pred_'+str(i)+'.txt', 'w')

    for k in range(len(ref_sentences)):
        input_ids = input_ids_list[k]
        input_len = len(input_ids)

        ori_tokens = []
        for m in range(len(ref_sentences[k])):
#             f_dev.write(ref_sentences[k][m]+'\n')
            ori_tokens.append(word_tokenize(ref_sentences[k][m]))
#         if k < len(ref_sentences)-1:
#             f_dev.write('\n')

        for _ in range(max_len):
            model_out = my_model.model_feeding(input_ids) # (batch, seq_len, emb_dim)
            pred_idx = model_out.argmax(1)[-1]        
            if pred_idx == my_model.tokenizer.eos_token_id:
                break            
            input_ids = torch.cat((input_ids, pred_idx.unsqueeze(0)), 0)        

        out_sen = my_model.tokenizer.decode(input_ids[input_len:])
        f_pred.write(out_sen+'\n')

        out_tokens = word_tokenize(out_sen)

        bleu_1_score = sentence_bleu(ori_tokens, out_tokens, weights=(1, 0, 0, 0))
        bleu_2_score = sentence_bleu(ori_tokens, out_tokens, weights=(0.5, 0.5, 0, 0))
        bleu_3_score = sentence_bleu(ori_tokens, out_tokens, weights=(0.33, 0.33, 0.33, 0))
        bleu_4_score = sentence_bleu(ori_tokens, out_tokens, weights=(0.25, 0.25, 0.25, 0.25))

        bleu_1 += bleu_1_score

        bleu_score += min(1, len(out_tokens)/len(ori_tokens))*((bleu_1_score*bleu_2_score*bleu_3_score*bleu_4_score)**(0.25))    

#     f_dev.close()
    f_pred.close()
    
    print(i, "th model")
    print("BLEU score: {}".format(bleu_score/len(ref_sentences)*100))
    print("BLEU1 score: {}".format(bleu_1/len(ref_sentences)*100))    

I0506 00:51:08.586743 139954174723904 file_utils.py:41] PyTorch version 1.2.0 available.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist

ok


4693it [00:02, 1630.54it/s]
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1 th model
BLEU score: 67.011002844619
BLEU1 score: 82.1452596341907
ok
2 th model
BLEU score: 74.80444465381424
BLEU1 score: 89.29666997331987
ok
3 th model
BLEU score: 72.97053596564822
BLEU1 score: 87.38864333690032
ok
4 th model
BLEU score: 72.98439792676945
BLEU1 score: 88.62888868036747
ok
5 th model
BLEU score: 74.13844074346744
BLEU1 score: 89.21020861290603
ok
6 th model
BLEU score: 74.42687041332428
BLEU1 score: 89.68754978933163
ok
7 th model
BLEU score: 74.26234161453121
BLEU1 score: 88.89235991750782
ok
8 th model
BLEU score: 73.61824480493395
BLEU1 score: 88.92714291178227


In [3]:
e2e_dataset = e2eDataset(csv_file='dataset/testset_w_refs.csv', tokenizer=my_model.tokenizer)
# dataloader = DataLoader(e2e_dataset, batch_size=1, shuffle=False, num_workers=4)
e2e_dataset[0]

NameError: name 'my_model' is not defined

## BERTscore

In [4]:
### BERT score with human reference
from bert_score import score
import glob
human_files = "/project/work/E2E/predictions/joosung2/testset/f_dev.txt"

human_open = open(human_files, "r")
human_dataset = human_open.readlines()
human_open.close()

human_references = []

temp_reference = []
for i in range(len(human_dataset)):
    if human_dataset[i] == '\n':
        human_references.append(temp_reference)
        temp_reference = []
    else:
        temp_reference.append(human_dataset[i].strip())
human_references.append(temp_reference)
human_compare = []
for i in range(len(human_references)):
    for k in range(len(human_references[i])):
        human_compare.append(human_references[i][k])

# output_path = "/project/work/E2E/predictions/joosung2/testset/base1_sample30/*"
output_path = "/project/work/E2E/predictions/final/*"
pred_files = glob.glob(output_path)
# pred_files = ["/project/work/E2E/predictions/final/sampling30_1.txt"]

score_list = []
for i in range(len(pred_files)):    
    cands = []
    pred_data_open = open(pred_files[i], "r")
    pred_data_dataset = pred_data_open.readlines()
    pred_len = len(pred_data_dataset)
    pred_data_open.close()
    
    for k in range(len(pred_data_dataset)):
        out_sen = pred_data_dataset[k].strip()
        repeat_num = len(human_references[k])
        for _ in range(repeat_num):
            cands.append(out_sen)

    P, R, F1 = score(cands, human_compare, lang='en', verbose=True)
    F1_list=list(F1.numpy())
    BERT_score = sum(F1_list)/len(F1_list)
    
    score_list.append(BERT_score)    

I0506 02:25:27.922973 140215530821440 configuration_utils.py:275] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-config.json from cache at /home/ds_user1/.cache/torch/transformers/c22e0b5bbb7c0cb93a87a2ae01263ae715b4c18d692b1740ce72cacaa99ad184.2d28da311092e99a05f9ee17520204614d60b0bfdb32f8a75644df7737b6a748
I0506 02:25:27.923840 140215530821440 configuration_utils.py:311] Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_ep

calculating scores...
computing bert embedding.



computing greedy matching.



done in 27.33 seconds, 171.71 sentences/sec


I0506 02:26:16.694179 140215530821440 configuration_utils.py:275] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-config.json from cache at /home/ds_user1/.cache/torch/transformers/c22e0b5bbb7c0cb93a87a2ae01263ae715b4c18d692b1740ce72cacaa99ad184.2d28da311092e99a05f9ee17520204614d60b0bfdb32f8a75644df7737b6a748
I0506 02:26:16.695167 140215530821440 configuration_utils.py:311] Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_ep

...


done in 28.88 seconds, 162.51 sentences/sec


I0506 02:31:01.515883 140215530821440 configuration_utils.py:275] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-config.json from cache at /home/ds_user1/.cache/torch/transformers/c22e0b5bbb7c0cb93a87a2ae01263ae715b4c18d692b1740ce72cacaa99ad184.2d28da311092e99a05f9ee17520204614d60b0bfdb32f8a75644df7737b6a748
I0506 02:31:01.516969 140215530821440 configuration_utils.py:311] Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_ep

calculating scores...
computing bert embedding.



computing greedy matching.



done in 28.80 seconds, 162.94 sentences/sec


In [5]:
pred_files, score_list

(['/project/work/E2E/predictions/final/final_model.txt',
  '/project/work/E2E/predictions/final/sampling10_3.txt',
  '/project/work/E2E/predictions/final/sampling30_1.txt',
  '/project/work/E2E/predictions/final/sampling10_1.txt',
  '/project/work/E2E/predictions/final/sampling50.txt',
  '/project/work/E2E/predictions/final/sampling30_3.txt',
  '/project/work/E2E/predictions/final/sampling30_2.txt',
  '/project/work/E2E/predictions/final/sampling10_2.txt',
  '/project/work/E2E/predictions/final/sampling30_4.txt'],
 [0.9420184408096827,
  0.939660089205964,
  0.9404677509270877,
  0.9385546076889615,
  0.9420467598235559,
  0.9393550385271541,
  0.9410023027687674,
  0.9391040466193982,
  0.9402433829776976])

In [ ]:
### Compared System / BERT score with human reference
import csv
from bert_score import score
import glob
human_files = "/project/work/E2E/predictions/testset/f_dev.txt"

human_open = open(human_files, "r")
human_dataset = human_open.readlines()
human_open.close()

human_references = []

temp_reference = []
for i in range(len(human_dataset)):
    if human_dataset[i] == '\n':
        human_references.append(temp_reference)
        temp_reference = []
    else:
        temp_reference.append(human_dataset[i].strip())
human_references.append(temp_reference)
human_compare = []
for i in range(len(human_references)):
    for k in range(len(human_references[i])):
        human_compare.append(human_references[i][k])

output_path = "/project/work/E2E/compared_system/system_outputs/primary_txt/*"
pred_files = glob.glob(output_path)

score_list = []
for i in range(len(pred_files)):    
    cands = []
    pred_data_open = open(pred_files[i], "r")
    pred_data_dataset = pred_data_open.readlines()
    pred_len = len(pred_data_dataset)
    pred_data_open.close()
    
    for k in range(len(pred_data_dataset)):
        out_sen = pred_data_dataset[k].strip()
        repeat_num = len(human_references[k])
        for _ in range(repeat_num):
            cands.append(out_sen)

    P, R, F1 = score(cands, human_compare, lang='en', verbose=True)
    F1_list=list(F1.numpy())
    BERT_score = sum(F1_list)/len(F1_list)
    
    score_list.append(BERT_score)  
print(pred_files, score_list)    

In [ ]:
for i in range(len(pred_files)):
    print(pred_files[i], score_list[i])

In [20]:
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_1.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_2.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_3.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_4.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_5.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_6.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_7.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_8.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_9.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/joosung2/testset/base2_sample30/f_pred_10.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/final/1base1_sample10_6.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/final/1base2_sample10_9.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/final/1base4_devtest_4.txt
# !./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt ./predictions/final/2base1_sample50_7.txt
!./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt /project/work/E2E/compared_system/system_outputs/primary_txt/slug.txt
!./e2e-metrics/measure_scores.py ./predictions/joosung2/testset/f_dev.txt /project/work/E2E/compared_system/system_outputs/primary_txt/slug-alt.txt

./predictions/joosung2/testset/f_dev.txt /project/work/E2E/compared_system/system_outputs/primary_txt/slug.txt None
Running MS-COCO evaluator...
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...
PTBTokenizer tokenized 129948 tokens at 547362.73 tokens per second.
PTBTokenizer tokenized 17228 tokens at 180441.17 tokens per second.
setting up scorers...
computing METEOR score...
METEOR: 0.445
computing Rouge score...
ROUGE_L: 0.677
computing CIDEr score...
CIDEr: 2.262
Creating temp directory  /tmp/e2e-eval-wbdld4sm
Running MTEval to compute BLEU & NIST...
Use of 'Hyphen' in \p{} or \P{} is deprecated because: Supplanted by Line_Break property values; see www.unicode.org/reports/tr14; at /project/work/E2E/e2e-metrics/mteval/mteval-v13a-sig.pl line 993.
MT evaluation scorer began on 2020 May 6 at 07:45:40
command line:  /project/work/E2E/e2e-metrics/mteval/mteval-v13a-sig.pl -r /tmp/e2e-eval-wbdld4sm/mtev

## Tsv to txt

In [ ]:
import pandas as pd
output_path = "/project/work/E2E/compared_system/system_outputs/primary/*"
comapred_files = glob.glob(output_path)

for i in range(len(comapred_files)):
    dataset = pd.read_csv(comapred_files[i], delimiter='\t', header=None)
    
    name = comapred_files[i].split('/')[-1].split('.')[0]
    txt_files = "/project/work/E2E/compared_system/system_outputs/primary_txt/"+name+".txt"
    f = open(txt_files, "w")
    gen_sentences = dataset[1]
    
    for k in range(1, len(gen_sentences)):
        f.write(gen_sentences[k]+'\n')
    f.close()

In [ ]:
len(gen_sentences)

## zero-shot test

In [ ]:
from model import *

max_len = 70
my_model = mymodel().cuda()
my_model.eval()
model_name = './gen_model/base_devtrain_4/4/model'
my_model.load_state_dict(torch.load(model_name))

In [ ]:
import random
import pandas as pd
csv_file='dataset/testset_w_refs.csv'
dataset = pd.read_csv(csv_file)
columns = dataset.columns
conditions = dataset[columns[0]]
        
typ_list = {}
for k in range(len(conditions)):
    cond_set = conditions[k].split(',')
    for m in range(len(cond_set)):
        cond_set[m] = cond_set[m].strip()
        pos = cond_set[m].index('[')
        if cond_set[m][:pos] in typ_list.keys():
            typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
        else:            
            typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}   

def sample_batch(tokenizer, cond_name, cond_set):
    condition_string = ''
    for m in range(len(cond_set)):
        condition_string += cond_name[m] + cond_set[m] + ' '

    input_string = condition_string + '<START>'
    input_ids = torch.tensor(tokenizer.encode(input_string, add_special_tokens=True))

    input_len = len(input_ids)

    return input_ids, condition_string

# <name> <eatType> <food> <priceRange> <customer rating> <area> <familyFriendly> <near>

name = None #'<NAME>'
eatType = None #'<EATTYPE>'
food = None # '<FOOD>'
priceRange = None # '<priceRange>'
customer_rating = None # '<CUSTOMER_RATING>'
area = '<area>' # None
familyFriendly = 'yes' # None
near = None #'<NEAR>'

cond_list = []
conditions = []
if name is not None:
    placeholder_name = random.choice(list(typ_list['name']))
    cond_list.append('<name>')
    conditions.append(placeholder_name)
if eatType is not None:
    placeholder_eatType = random.choice(list(typ_list['eatType']))
    cond_list.append('<eatType>')
    conditions.append(placeholder_eatType)
if food is not None:
    placeholder_food = random.choice(list(typ_list['food']))
    cond_list.append('<food>')
    conditions.append(placeholder_food)
if priceRange is not None:
    placeholder_priceRange = random.choice(list(typ_list['priceRange']))
    cond_list.append('<priceRange>')
    conditions.append(placeholder_priceRange)    
if customer_rating is not None:
    placeholder_customer_rating = random.choice(list(typ_list['customer rating']))
    cond_list.append('<customer rating>')
    conditions.append(placeholder_customer_rating)        
if area is not None:
    placeholder_area = random.choice(list(typ_list['area']))
    cond_list.append('<area>')
    conditions.append(placeholder_area)    
if familyFriendly is not None:
    cond_list.append('<familyFriendly>')
    conditions.append(familyFriendly)            
if near is not None:
    placeholder_near = random.choice(list(typ_list['near']))
    cond_list.append('<near>')
    conditions.append(placeholder_near)        


# del cond_name[2]
# del conditions[2]

sample = sample_batch(my_model.tokenizer, cond_list, conditions)

input_ids = sample[0].cuda()
condition_string = sample[1]  
input_len = len(input_ids)

max_len = 70
for _ in range(max_len):
    model_out = my_model.model_feeding(input_ids) # (batch, seq_len, emb_dim)
    pred_idx = model_out.argmax(1)[-1]        
    if pred_idx == my_model.tokenizer.eos_token_id:
        break            
    input_ids = torch.cat((input_ids, pred_idx.unsqueeze(0)), 0)        

out_sen = my_model.tokenizer.decode(input_ids[input_len:])
print(cond_list)
print(conditions)
print(out_sen)

if name is not None:
    out_sen = out_sen.replace(placeholder_name, name)
if eatType is not None:
    out_sen = out_sen.replace(placeholder_eatType, eatType)
if food is not None:
    out_sen = out_sen.replace(placeholder_food, food)
if priceRange is not None:
    out_sen = out_sen.replace(placeholder_priceRange, priceRange)
if customer_rating is not None:
    out_sen = out_sen.replace(placeholder_customer_rating, customer_rating)
if area is not None:
    out_sen = out_sen.replace(placeholder_area, area)  
if near is not None:
    out_sen = out_sen.replace(placeholder_near, near)
print(out_sen)

In [ ]:
condition_string

## delexicalization

In [ ]:
class e2eDataset(Dataset):
    def __init__(self, csv_file1, csv_file2, tokenizer):
        """
        Args:
            csv_file (string): csv 파일의 경로
        """
        self.dataset1 = pd.read_csv(csv_file1)
        self.dataset2 = pd.read_csv(csv_file2)
        
        self.columns1 = self.dataset1.columns
        self.columns2 = self.dataset2.columns
        
        self.conditions = list(self.dataset1[self.columns1[0]]) + list(self.dataset2[self.columns2[0]])
        self.sentences = list(self.dataset1[self.columns1[1]]) + list(self.dataset2[self.columns2[1]])
        self.tokenizer = tokenizer
        
        self.typ_list = {}
        for k in range(len(self.conditions)):
            cond_set = self.conditions[k].split(',')
            for m in range(len(cond_set)):
                cond_set[m] = cond_set[m].strip()
                pos = cond_set[m].index('[')
                if cond_set[m][:pos] in self.typ_list.keys():
                    self.typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
                else:            
                    self.typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}        

    def __len__(self):
        return len(self.conditions)

    def __getitem__(self, idx):
        sen = self.sentences[idx]
        
        cond = self.conditions[idx]
        cond_set = cond.split(',')        
        condition_string = ''
        
        
        p = random.random()

        if p > 0.3: # 70%
            for m in range(len(cond_set)):
                cond_set[m] = cond_set[m].strip()
                pos = cond_set[m].index('[')

                condition_string += '<' + cond_set[m][:pos] + '>' + cond_set[m][pos+1:-1] + ' '
        else: # p <= 0.3 / 30%
            nochange_list = ['priceRange', 'customer rating', 'familyFriendly']
            for m in range(len(cond_set)):
                cond_set[m] = cond_set[m].strip()
                
                pos = cond_set[m].index('[')
                if cond_set[m][:pos] not in nochange_list:
                    placeholder = '<' + cond_set[m][:pos] + '>'
                    condition_string += placeholder + ' '
                    sen = sen.replace(cond_set[m][pos+1:-1], placeholder)        
                else:
                    condition_string += '<' + cond_set[m][:pos] + '>' + cond_set[m][pos+1:-1] + ' '
        
        input_string = condition_string + '<START>' + sen
        input_ids = torch.tensor(self.tokenizer.encode(input_string, add_special_tokens=True))
        
        label_string = sen + ' <|endoftext|>'
        label_ids = torch.tensor(self.tokenizer.encode(label_string, add_special_tokens=True))

        return input_string, input_ids, label_ids    

In [ ]:
e2e_dataset = e2eDataset(csv_file1='dataset/trainset.csv', csv_file2='dataset/devset.csv', tokenizer=my_model.tokenizer)
print(e2e_dataset[200][0])
# print(e2e_dataset.typ_list.keys())
# print(e2e_dataset.typ_list)
# priceRange, customer rating, familyFriendly

# check data

In [ ]:
f_dev = open('f_dev.txt', 'r')
f_pred = open('f_pred.txt', 'r')
f_dev_dataset = f_dev.readlines()
f_pred_dataset = f_pred.readlines()

In [ ]:
len(f_dev_dataset), len(f_pred_dataset)

In [ ]:
data=[x for x in f_dev_dataset if x != '\n']
data2=[x for x in f_pred_dataset if x != '\n']
len(data), len(data2)

In [ ]:
4672+546

In [ ]:
f_dev = open('./e2e-metrics/example-inputs/devel-conc.txt', 'r')
f_pred = open('./e2e-metrics/example-inputs/baseline-output.txt', 'r')
f_dev_dataset = f_dev.readlines()
f_pred_dataset = f_pred.readlines()

In [ ]:
len(f_dev_dataset), len(f_pred_dataset)

In [ ]:
data=[x for x in f_dev_dataset if x != '\n']
data2=[x for x in f_pred_dataset if x != '\n']
len(data), len(data2)

In [ ]:
import random
p=random.random()
p

## Ouput 비교

In [34]:
import csv
import pandas as pd
csv_file="/project/work/E2E/dataset/testset_w_refs.csv"
dataset = pd.read_csv(csv_file)
columns = dataset.columns
conditions = dataset[columns[0]]
refs = dataset[columns[1]]

ref_conditions = []
ref_sentences = []
pre_condition = ''
start = 0
for k in range(len(refs)):
    sen = refs[k]    
        
    if start == 0 or conditions[k] == pre_condition:      
        if start == 0:
            ref_sentences.append(sen)
            ref_conditions.append(conditions[k])
        pre_condition = conditions[k]
        start += 1
    else:
        ref_sentences.append(sen)
        ref_conditions.append(conditions[k])
        pre_condition = conditions[k]
        start += 1        


output_path = "/project/work/E2E/compared_system/system_outputs/compared/"
pred_files = [output_path+'our_model.txt',output_path+'slug.txt',output_path+'tgen.txt', output_path+'tuda.txt']

sentence_list = []
file_name = ['our_model', 'slug', 'tgen', 'tuda', 'references']
for i in range(len(pred_files)):
    cands = []
    pred_data_open = open(pred_files[i], "r")
    pred_data_dataset = pred_data_open.readlines()
    pred_len = len(pred_data_dataset)
    pred_data_open.close()
    
    for k in range(len(pred_data_dataset)):
        out_sen = pred_data_dataset[k].strip()
        cands.append(out_sen)
    
    sentence_list.append(cands)  
sentence_list.append(ref_sentences)

In [48]:
f = open(output_path + 'total.txt', 'w')

print(file_name)

for ind in range(630):
#     if sentence_list[0][ind] != sentence_list[-1][ind]:
    f.write(str(ind) + '\t' + ref_conditions[ind] + '\n')
    print(ind, ref_conditions[ind])
    for k in range(len(file_name)):
        print(sentence_list[k][ind])
        f.write(sentence_list[k][ind] + '\n')
f.close()

['our_model', 'slug', 'tgen', 'tuda', 'references']
0 name[Blue Spice], eatType[coffee shop], area[city centre]
Blue Spice is a coffee shop located in the city centre.
Blue Spice is a coffee shop in the city centre.
Blue Spice is a coffee shop in the city centre.
Blue Spice is a coffee shop located in the city centre area.
A coffee shop in the city centre area called Blue Spice.
1 name[Blue Spice], eatType[coffee shop], area[riverside]
Blue Spice is a coffee shop in the riverside area.
Blue Spice is a coffee shop in the riverside area.
Blue Spice is a coffee shop in the riverside area with a price range of less than £20.
Blue Spice is a coffee shop located in the riverside area.
There is a coffee shop Blue Spice in the riverside area.
2 name[Blue Spice], eatType[coffee shop], customer rating[5 out of 5], near[Crowne Plaza Hotel]
Blue Spice is a coffee shop near Crowne Plaza Hotel with a customer rating of 5 out of 5.
Blue Spice is a coffee shop near Crowne Plaza Hotel with a customer r

In [40]:
if sentence_list[0][ind] != sentence_list[-1][ind]:
    print(sentence_list[0][ind], sentence_list[-1][ind])

Zizzi is a pub near The Sorrento. Close to The Sorrento you will be able to find a pub called Zizzi
